# Tiền xử lý dữ liệu

Mục tiêu: chúng ta load được các dữ liệu từ các file JSON, sau đó sẽ loại bỏ:
1. Cleaning: Loại bỏ các dấu câu đã được chuyển thành NCR (numeric character reference)
2. Tokenizer -> sau đó loại bỏ Stopwords
3. \[Optional\] Chuyển từ viết tắt thành từ đầy đủ


In [16]:
import os,json
import pandas as pd

import unicodedata as uni
import sys

import html

_json&#95;files_: List chứa các file JSON trong data


In [2]:
path_to_news = "Data/"
json_files = [pos_json for pos_json in os.listdir(path_to_news) if pos_json.endswith('.json')]

In [3]:
print(json_files)

['real_1.json', 'fake_1.json']


Tạo dataframe bằng [pandas.DataFrame](http://pandas-docs.github.io/pandas-docs-travis/generated/pandas.Series.loc.html) để có thể lưu các thuộc tính load từ các file JSON đã được load ở trên

In [4]:
news = pd.DataFrame(columns=['title', 'text', 'authors'])

Load dữ liệu từ các file JSON vào list news theo các thuộc tính: _title_, _text_ và _authors_

In [5]:
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_news, js), 'r', encoding='utf-8') as json_file:
        json_text = json.load(json_file)
        
        
        title = json_text['title']
        text = json_text['text']
        author = json_text['authors']
        
        news.loc[index] = [title, text, author]

## 1. Loại bỏ dấu câu

Trong dữ liệu sau khi được crawl về, các ký tự đặc biệt (ví dụ các dấu câu) sẽ được đổi thành dạng [numeric character reference (ncr)](https://en.wikipedia.org/wiki/Numeric_character_reference). Ví dụ như: Trong file _real&#95;1.json_ phần title (ở bên dưới). 

Chúng ta phải đổi lại dưới dạng Unicode để có thể loại bỏ phần dấu câu. 

_html.unescape(str)_: đổi các kí tự dạng __ncr__ về định dạng Unicode của chúng

Tham khảo [stackoverflow](https://stackoverflow.com/a/17018874/5144980)

In [6]:
title_str = news.loc[0]['title']

title_str = html.unescape(title_str)

_remove&#95;punc_: hàm dùng để loại bỏ các kí tự đặc biệt như dấu câu trong định dạng Unicode. Tham khảo: 

In [17]:
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                       if uni.category(chr(i)).startswith('P'))

def remove_punc(text):
    return text.translate(tbl)

__Ví dụ:__ Title của _real&#95;1.json_ 

In [8]:
print(title_str)

Tung tin 'Mr Bean' qua đời để phát tán virus độc hại


Title của _real&#95;1.json_ sau khi loại bỏ kí tự đặc biệt: 

In [9]:
new_string = remove_punc(title_str)
print(new_string)

Tung tin Mr Bean qua đời để phát tán virus độc hại


### Solution

Bắt đầu xử lý các dữ liệu trong list các file JSON và lưu trong list ___preproc&#95;news___:

In [11]:
preproc_news = pd.DataFrame(columns=['title', 'text', 'authors'])

for index, js in enumerate(json_files):
    with open(os.path.join(path_to_news, js), 'r', encoding='utf-8') as json_file:
        json_text = json.load(json_file)
        
        
        title = remove_punc(html.unescape(json_text['title']))
        text = remove_punc(html.unescape(json_text['text']))
        author = json_text['authors']
        
        preproc_news.loc[index] = [title, text, author]
        
print(preproc_news.loc[0].text)

Hôm 197 nhiều độc giả cảnh báo nhau trên mạng về một loại virus máy tính phát tán qua tin giả Mr Bean  Rowan Atkinson qua đời mạo danh Fox News

Thông tin lan truyền cách đây ít phút trên mạng xã hội Theo trang SDE các đối tượng phát tán những bài đăng giả trên mạng xã hội với hình thức như một đường link tin tức tiêu đề là Tin nóng của Fox News Mr Bean Rowan Atkinson vừa qua đời ở tuổi 62 sau một vụ đâm xe trong lúc đóng cảnh hành động mạo hiểm Hôm nay tháng 72017
Tung tin Mr Bean qua doi de phat tan virus doc hai hinh anh 1
Tin cải chính về cái chết của Rowan Atkinson được đưa ra kịp thời để bảo vệ người dùng khỏi virus Ảnh Chronicle Live 

Vì gắn với một cơ quan truyền thông lớn là Fox News nên mẩu tin dễ dàng thu hút sự chú ý và tin cậy của độc giả dù thông tin ngày tháng bị sai lệch Trang SDE cảnh báo bất cứ độc giả nào nhìn thấy đường link chứa thông tin này không được bấm vào vì thực chất đó là một virus đội lốt

Đường link sẽ dẫn thẳng đến một trang web lừa đảo và mã độc nhảy v

## 2. Tokenizer & Loại StopWords

Trước hết cần biểu diễn token sau đó loại bỏ đi các stopword

Dữ liệu từ điển được lấy từ (các file được chứa trong thư mục ./corpus):
1. [deepai-solutions/core_nlp](https://github.com/deepai-solutions/core_nlp/tree/master/tokenization): 2 file _bi_\__gram.txt_ và _tri_\__gram.txt_

### 2.1. Cách 1: Sử dụng so khớp dài nhất 

Sử dụng so khớp dài nhất (longest matching) để tách các từ trong  tin tức vừa loại dấu câu

In [2]:
import ast

In [5]:
def load_n_gram(file_path):
    with open(file_path, encoding='utf-8') as n_gram:
        words = n_gram.read()
        words = ast.literal_eval(words)
        return words

In [9]:
corpus_path = './corpus/'
bi_gram_path = 'bi_gram.txt'
tri_gram_path = 'tri_gram.txt'

In [10]:
bi_gram = load_n_gram(os.path.join(corpus_path, bi_gram_path))
tri_gram = load_n_gram(os.path.join(corpus_path, tri_gram_path))

In [14]:
print (len(bi_gram))

22705


In [ ]:
def LongestMatching(text, n_gram):
    